In [1]:
!pip install librosa pandas tensorflow scikit-learn matplotlib

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-10.4.0-cp38-cp38-win_amd64.whl.metadata (9.3 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl (9.3 MB)
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   -------- ------------------------------- 1.6/7.5 MB 9.3 MB/s eta 0:00:01
   ------------------- -------------------- 3.7/7

In [2]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import shutil
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout, Input

In [3]:
zip_path = "archive (3).zip"
extract_path = "extracted_dataset"

In [4]:
shutil.unpack_archive(zip_path, extract_path)

In [5]:
dataset_path = extract_path

In [6]:
def extract_features(file_path, max_pad_length=174):
    try:
        audio, sample_rate = librosa.load(file_path, sr=22050)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, max(0, pad_width))), mode='constant')
        return mfccs.T  # Transposing to (timesteps, features)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [7]:
audio_files = []
labels = []

In [8]:
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(file_path))  # Use parent folder name as label
            features = extract_features(file_path)
            if features is not None:
                audio_files.append(features)
                labels.append(label)

In [9]:
# Convert to numpy arrays
if audio_files:
    audio_features = np.array(audio_files)
    labels = np.array(labels)
else:
    raise ValueError("No audio files were loaded. Check dataset path and file format.")

In [10]:
# Dynamically detect emotion categories
emotion_categories = sorted(set(labels))  # Extract unique labels from dataset
encoder = LabelEncoder()
encoder.fit(emotion_categories)
y = encoder.transform(labels)

In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(audio_features, y, test_size=0.2, random_state=42)

In [12]:
# Adjust input shape based on extracted features
input_shape = X_train.shape[1:]

In [13]:
# Build Model
model = Sequential([
    Input(shape=input_shape),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(emotion_categories), activation='softmax')
])

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train Model
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/25
194/194 [==============================] - 81s 339ms/step - loss: 0.8760 - accuracy: 0.6786 - val_loss: 0.7570 - val_accuracy: 0.7414
Epoch 2/25
194/194 [==============================] - 64s 330ms/step - loss: 0.6675 - accuracy: 0.7558 - val_loss: 0.5633 - val_accuracy: 0.7841
Epoch 3/25
194/194 [==============================] - 57s 294ms/step - loss: 0.4871 - accuracy: 0.8211 - val_loss: 0.4402 - val_accuracy: 0.8145
Epoch 4/25
194/194 [==============================] - 53s 272ms/step - loss: 0.4053 - accuracy: 0.8529 - val_loss: 0.4002 - val_accuracy: 0.8455
Epoch 5/25
194/194 [==============================] - 49s 252ms/step - loss: 0.3506 - accuracy: 0.8715 - val_loss: 0.4261 - val_accuracy: 0.8436
Epoch 6/25
194/194 [==============================] - 49s 251ms/step - loss: 0.3083 - accuracy: 0.8891 - val_loss: 0.3294 - val_accuracy: 0.8798
Epoch 7/25
194/194 [==============================] - 47s 243ms/step - loss: 0.2755 - accuracy: 0.9008 - val_loss: 0.3362 - val_ac

In [16]:
# Save Model
model.save("mental_health_emotion_model.keras")